# Column and vector operations

Column and vector operations were added in 1.15.

* cols2vec
* vec2cols
* drop columns

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [26]:
# Greenplum Database 5.4.0 on GCP (demo machine)
%sql postgresql://gpadmin@35.184.253.255:5432/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

u'Connected: gpadmin@madlib'

In [27]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.15-dev, git revision: rc/1.14-rc1-52-g1a7c756, cmake configuration time: Tue Jul 31 20:31:52 UTC 2018, build type: release, build system: Linux-2.6.32-696.20.1.el6.x86_64, C compiler: gcc 4.4.7, C++ compiler: g++ 4.4.7"


# Cols2vec

Convert feature columns in a table into an array in a single column.

Given a table with a number of feature columns, this function will create an output table that contains the feature columns in an array. A summary table will also be created that contains the names of the features combined into array, so that this process can be reversed using the function vec2cols.

# 1. Load sample data

In [28]:
%%sql
DROP TABLE IF EXISTS golf CASCADE;

CREATE TABLE golf (
    id integer NOT NULL,
    "OUTLOOK" text,
    temperature double precision,
    humidity double precision,
    "Temp_Humidity" double precision[],
    clouds_airquality text[],
    windy boolean,
    class text,
    observation_weight double precision
);

INSERT INTO golf VALUES
(1,'sunny', 85, 85, ARRAY[85, 85],ARRAY['none', 'unhealthy'], 'false','Don''t Play', 5.0),
(2, 'sunny', 80, 90, ARRAY[80, 90], ARRAY['none', 'moderate'], 'true', 'Don''t Play', 5.0),
(3, 'overcast', 83, 78, ARRAY[83, 78], ARRAY['low', 'moderate'], 'false', 'Play', 1.5),
(4, 'rain', 70, 96, ARRAY[70, 96], ARRAY['low', 'moderate'], 'false', 'Play', 1.0),
(5, 'rain', 68, 80, ARRAY[68, 80], ARRAY['medium', 'good'], 'false', 'Play', 1.0),
(6, 'rain', 65, 70, ARRAY[65, 70], ARRAY['low', 'unhealthy'], 'true', 'Don''t Play', 1.0),
(7, 'overcast', 64, 65, ARRAY[64, 65], ARRAY['medium', 'moderate'], 'true', 'Play', 1.5),
(8, 'sunny', 72, 95, ARRAY[72, 95], ARRAY['high', 'unhealthy'], 'false', 'Don''t Play', 5.0),
(9, 'sunny', 69, 70, ARRAY[69, 70], ARRAY['high', 'good'], 'false', 'Play', 5.0),
(10, 'rain', 75, 80, ARRAY[75, 80], ARRAY['medium', 'good'], 'false', 'Play', 1.0),
(11, 'sunny', 75, 70, ARRAY[75, 70], ARRAY['none', 'good'], 'true', 'Play', 5.0),
(12, 'overcast', 72, 90, ARRAY[72, 90], ARRAY['medium', 'moderate'], 'true', 'Play', 1.5),
(13, 'overcast', 81, 75, ARRAY[81, 75], ARRAY['medium', 'moderate'], 'false', 'Play', 1.5),
(14, 'rain', 71, 80, ARRAY[71, 80], ARRAY['low', 'unhealthy'], 'true', 'Don''t Play', 1.0);

SELECT * FROM golf ORDER BY id;

Done.
Done.
14 rows affected.
14 rows affected.


id,OUTLOOK,temperature,humidity,Temp_Humidity,clouds_airquality,windy,class,observation_weight
1,sunny,85.0,85.0,"[85.0, 85.0]","[u'none', u'unhealthy']",False,Don't Play,5.0
2,sunny,80.0,90.0,"[80.0, 90.0]","[u'none', u'moderate']",True,Don't Play,5.0
3,overcast,83.0,78.0,"[83.0, 78.0]","[u'low', u'moderate']",False,Play,1.5
4,rain,70.0,96.0,"[70.0, 96.0]","[u'low', u'moderate']",False,Play,1.0
5,rain,68.0,80.0,"[68.0, 80.0]","[u'medium', u'good']",False,Play,1.0
6,rain,65.0,70.0,"[65.0, 70.0]","[u'low', u'unhealthy']",True,Don't Play,1.0
7,overcast,64.0,65.0,"[64.0, 65.0]","[u'medium', u'moderate']",True,Play,1.5
8,sunny,72.0,95.0,"[72.0, 95.0]","[u'high', u'unhealthy']",False,Don't Play,5.0
9,sunny,69.0,70.0,"[69.0, 70.0]","[u'high', u'good']",False,Play,5.0
10,rain,75.0,80.0,"[75.0, 80.0]","[u'medium', u'good']",False,Play,1.0


# 2. Create feature array
Combine the temperature and humidity columns into a single array feature.

In [5]:
%%sql
DROP TABLE IF EXISTS cols2vec_result, cols2vec_result_summary;

SELECT madlib.cols2vec(
    'golf',
    'cols2vec_result',
    'temperature, humidity'
);

SELECT * FROM cols2vec_result;

Done.
1 rows affected.
14 rows affected.


feature_vector
"[85.0, 85.0]"
"[68.0, 80.0]"
"[69.0, 70.0]"
"[81.0, 75.0]"
"[80.0, 90.0]"
"[65.0, 70.0]"
"[75.0, 80.0]"
"[71.0, 80.0]"
"[83.0, 78.0]"
"[64.0, 65.0]"


View the summary table:

In [8]:
%%sql
SELECT * FROM cols2vec_result_summary;

1 rows affected.


source_table,list_of_features,list_of_features_to_exclude,feature_names
golf,"temperature, humidity",None,"[u'temperature', u'humidity']"


# 3.  Create feature array and keep some source table columns
Combine the temperature and humidity columns and keep 2 other columns from source_table.

In [7]:
%%sql
DROP TABLE IF EXISTS cols2vec_result, cols2vec_result_summary;

SELECT madlib.cols2vec(
    'golf',
    'cols2vec_result',
    'temperature, humidity',
    NULL,
    'id, "OUTLOOK"'
);

SELECT * FROM cols2vec_result ORDER BY id;

Done.
1 rows affected.
14 rows affected.


id,OUTLOOK,feature_vector
1,sunny,"[85.0, 85.0]"
2,sunny,"[80.0, 90.0]"
3,overcast,"[83.0, 78.0]"
4,rain,"[70.0, 96.0]"
5,rain,"[68.0, 80.0]"
6,rain,"[65.0, 70.0]"
7,overcast,"[64.0, 65.0]"
8,sunny,"[72.0, 95.0]"
9,sunny,"[69.0, 70.0]"
10,rain,"[75.0, 80.0]"


View the summary table:

In [9]:
%%sql
SELECT * FROM cols2vec_result_summary;

1 rows affected.


source_table,list_of_features,list_of_features_to_exclude,feature_names
golf,"temperature, humidity",None,"[u'temperature', u'humidity']"


# 4. Exclude all columns that are not double precision
Combine all columns, excluding all columns that are not of type double precision

In [10]:
%%sql
DROP TABLE IF EXISTS cols2vec_result, cols2vec_result_summary;

SELECT madlib.cols2vec(
    'golf',
    'cols2vec_result',
    '*',
    '"OUTLOOK", "Temp_Humidity", clouds_airquality, windy, class, id',
    'id, "OUTLOOK"'
);

SELECT * FROM cols2vec_result ORDER BY id;

Done.
1 rows affected.
14 rows affected.


id,OUTLOOK,feature_vector
1,sunny,"[85.0, 85.0, 5.0]"
2,sunny,"[80.0, 90.0, 5.0]"
3,overcast,"[83.0, 78.0, 1.5]"
4,rain,"[70.0, 96.0, 1.0]"
5,rain,"[68.0, 80.0, 1.0]"
6,rain,"[65.0, 70.0, 1.0]"
7,overcast,"[64.0, 65.0, 1.5]"
8,sunny,"[72.0, 95.0, 5.0]"
9,sunny,"[69.0, 70.0, 5.0]"
10,rain,"[75.0, 80.0, 1.0]"


In [11]:
%%sql
SELECT * FROM cols2vec_result_summary;

1 rows affected.


source_table,list_of_features,list_of_features_to_exclude,feature_names
golf,*,"""OUTLOOK"", ""Temp_Humidity"", clouds_airquality, windy, class, id","[u'temperature', u'humidity', u'observation_weight']"


# 5. Keep all columns from source table
Combine the temperature and humidity columns, exclude windy, and keep all of the columns from the source table.

In [12]:
%%sql
DROP TABLE IF EXISTS cols2vec_result, cols2vec_result_summary;

SELECT madlib.cols2vec(
    'golf',
    'cols2vec_result',
    'windy, temperature, humidity',
    'windy',
    '*'
);

SELECT * FROM cols2vec_result ORDER BY id;

Done.
1 rows affected.
14 rows affected.


id,OUTLOOK,temperature,humidity,Temp_Humidity,clouds_airquality,windy,class,observation_weight,feature_vector
1,sunny,85.0,85.0,"[85.0, 85.0]","[u'none', u'unhealthy']",False,Don't Play,5.0,"[85.0, 85.0]"
2,sunny,80.0,90.0,"[80.0, 90.0]","[u'none', u'moderate']",True,Don't Play,5.0,"[80.0, 90.0]"
3,overcast,83.0,78.0,"[83.0, 78.0]","[u'low', u'moderate']",False,Play,1.5,"[83.0, 78.0]"
4,rain,70.0,96.0,"[70.0, 96.0]","[u'low', u'moderate']",False,Play,1.0,"[70.0, 96.0]"
5,rain,68.0,80.0,"[68.0, 80.0]","[u'medium', u'good']",False,Play,1.0,"[68.0, 80.0]"
6,rain,65.0,70.0,"[65.0, 70.0]","[u'low', u'unhealthy']",True,Don't Play,1.0,"[65.0, 70.0]"
7,overcast,64.0,65.0,"[64.0, 65.0]","[u'medium', u'moderate']",True,Play,1.5,"[64.0, 65.0]"
8,sunny,72.0,95.0,"[72.0, 95.0]","[u'high', u'unhealthy']",False,Don't Play,5.0,"[72.0, 95.0]"
9,sunny,69.0,70.0,"[69.0, 70.0]","[u'high', u'good']",False,Play,5.0,"[69.0, 70.0]"
10,rain,75.0,80.0,"[75.0, 80.0]","[u'medium', u'good']",False,Play,1.0,"[75.0, 80.0]"


View summary table:

In [14]:
%%sql
SELECT * FROM cols2vec_result_summary;

1 rows affected.


source_table,list_of_features,list_of_features_to_exclude,feature_names
golf,"windy, temperature, humidity",windy,"[u'temperature', u'humidity']"


This also shows that you can exclude features in 'list_of_features_to_exclude' that are in the list of 'list_of_features'.  This can be useful if the 'list_of_features' is generated from an expression or subquery.

# 6. Boolean casting

Type casting works as per regular rules of the underlying database.  E.g, combining integer and double precisions columns will create a double precision feature vector.  For Boolean, do an explicit cast to the target type:

In [30]:
%%sql
DROP TABLE IF EXISTS cols2vec_result, cols2vec_result_summary;

SELECT madlib.cols2vec(
    'golf',
    'cols2vec_result',
    'windy::TEXT, class',
    NULL,
    'id'
);

SELECT * FROM cols2vec_result ORDER BY id;

Done.
1 rows affected.
14 rows affected.


id,feature_vector
1,"[u'false', u""Don't Play""]"
2,"[u'true', u""Don't Play""]"
3,"[u'false', u'Play']"
4,"[u'false', u'Play']"
5,"[u'false', u'Play']"
6,"[u'true', u""Don't Play""]"
7,"[u'true', u'Play']"
8,"[u'false', u""Don't Play""]"
9,"[u'false', u'Play']"
10,"[u'false', u'Play']"


# Vec2cols

Converts a feature array in a single column into multiple columns. This process can be used to reverse the function cols2vec.

Given a table with a column of type array, this function will create an output table that splits this array into multiple columns, one per array element. It includes the option to name the new feature columns, and to include columns from the original table in the output.

# 1. Load sample data

In [18]:
%%sql
DROP TABLE IF EXISTS golf CASCADE;

CREATE TABLE golf (
    id integer NOT NULL,
    "OUTLOOK" text,
    temperature double precision,
    humidity double precision,
    "Temp_Humidity" double precision[],
    clouds_airquality text[],
    windy boolean,
    class text,
    observation_weight double precision
);

INSERT INTO golf VALUES
(1,'sunny', 85, 85, ARRAY[85, 85],ARRAY['none', 'unhealthy'], 'false','Don''t Play', 5.0),
(2, 'sunny', 80, 90, ARRAY[80, 90], ARRAY['none', 'moderate'], 'true', 'Don''t Play', 5.0),
(3, 'overcast', 83, 78, ARRAY[83, 78], ARRAY['low', 'moderate'], 'false', 'Play', 1.5),
(4, 'rain', 70, 96, ARRAY[70, 96], ARRAY['low', 'moderate'], 'false', 'Play', 1.0),
(5, 'rain', 68, 80, ARRAY[68, 80], ARRAY['medium', 'good'], 'false', 'Play', 1.0),
(6, 'rain', 65, 70, ARRAY[65, 70], ARRAY['low', 'unhealthy'], 'true', 'Don''t Play', 1.0),
(7, 'overcast', 64, 65, ARRAY[64, 65], ARRAY['medium', 'moderate'], 'true', 'Play', 1.5),
(8, 'sunny', 72, 95, ARRAY[72, 95], ARRAY['high', 'unhealthy'], 'false', 'Don''t Play', 5.0),
(9, 'sunny', 69, 70, ARRAY[69, 70], ARRAY['high', 'good'], 'false', 'Play', 5.0),
(10, 'rain', 75, 80, ARRAY[75, 80], ARRAY['medium', 'good'], 'false', 'Play', 1.0),
(11, 'sunny', 75, 70, ARRAY[75, 70], ARRAY['none', 'good'], 'true', 'Play', 5.0),
(12, 'overcast', 72, 90, ARRAY[72, 90], ARRAY['medium', 'moderate'], 'true', 'Play', 1.5),
(13, 'overcast', 81, 75, ARRAY[81, 75], ARRAY['medium', 'moderate'], 'false', 'Play', 1.5),
(14, 'rain', 71, 80, ARRAY[71, 80], ARRAY['low', 'unhealthy'], 'true', 'Don''t Play', 1.0);

SELECT * FROM golf ORDER BY id;

Done.
Done.
14 rows affected.
14 rows affected.


id,OUTLOOK,temperature,humidity,Temp_Humidity,clouds_airquality,windy,class,observation_weight
1,sunny,85.0,85.0,"[85.0, 85.0]","[u'none', u'unhealthy']",False,Don't Play,5.0
2,sunny,80.0,90.0,"[80.0, 90.0]","[u'none', u'moderate']",True,Don't Play,5.0
3,overcast,83.0,78.0,"[83.0, 78.0]","[u'low', u'moderate']",False,Play,1.5
4,rain,70.0,96.0,"[70.0, 96.0]","[u'low', u'moderate']",False,Play,1.0
5,rain,68.0,80.0,"[68.0, 80.0]","[u'medium', u'good']",False,Play,1.0
6,rain,65.0,70.0,"[65.0, 70.0]","[u'low', u'unhealthy']",True,Don't Play,1.0
7,overcast,64.0,65.0,"[64.0, 65.0]","[u'medium', u'moderate']",True,Play,1.5
8,sunny,72.0,95.0,"[72.0, 95.0]","[u'high', u'unhealthy']",False,Don't Play,5.0
9,sunny,69.0,70.0,"[69.0, 70.0]","[u'high', u'good']",False,Play,5.0
10,rain,75.0,80.0,"[75.0, 80.0]","[u'medium', u'good']",False,Play,1.0


# 2. Split array into columns
Split the column "clouds_airquality" into new columns called "clouds" and "air_quality". Also keep columns id and "OUTLOOK" from the source table.

In [19]:
%%sql
DROP TABLE IF EXISTS vec2cols_result;

SELECT madlib.vec2cols(
    'golf',                              -- source table
    'vec2cols_result',                   -- output table
    'clouds_airquality',                 -- column with array entries to split
    ARRAY['clouds', 'air_quality'],      -- feature names
    'id, "OUTLOOK", '                    -- columns to keep from source table
);

SELECT * FROM vec2cols_result ORDER BY id;

Done.
1 rows affected.
14 rows affected.


id,OUTLOOK,clouds,air_quality
1,sunny,none,unhealthy
2,sunny,none,moderate
3,overcast,low,moderate
4,rain,low,moderate
5,rain,medium,good
6,rain,low,unhealthy
7,overcast,medium,moderate
8,sunny,high,unhealthy
9,sunny,high,good
10,rain,medium,good


# 3. Auto-generate feature names
Similar to the previous example, except now we keep all columns from source table and do not specify the feature names, so that default names are created.

In [20]:
%%sql
DROP TABLE IF EXISTS vec2cols_result;
SELECT madlib.vec2cols(
    'golf',                       -- source table
    'vec2cols_result',            -- output table
    'clouds_airquality',          -- column with array entries to split
    NULL,                         -- feature names
    '*'                           -- columns to keep from source table
);
SELECT * FROM vec2cols_result ORDER BY id;

Done.
1 rows affected.
14 rows affected.


id,OUTLOOK,temperature,humidity,Temp_Humidity,clouds_airquality,windy,class,observation_weight,f1,f2
1,sunny,85.0,85.0,"[85.0, 85.0]","[u'none', u'unhealthy']",False,Don't Play,5.0,none,unhealthy
2,sunny,80.0,90.0,"[80.0, 90.0]","[u'none', u'moderate']",True,Don't Play,5.0,none,moderate
3,overcast,83.0,78.0,"[83.0, 78.0]","[u'low', u'moderate']",False,Play,1.5,low,moderate
4,rain,70.0,96.0,"[70.0, 96.0]","[u'low', u'moderate']",False,Play,1.0,low,moderate
5,rain,68.0,80.0,"[68.0, 80.0]","[u'medium', u'good']",False,Play,1.0,medium,good
6,rain,65.0,70.0,"[65.0, 70.0]","[u'low', u'unhealthy']",True,Don't Play,1.0,low,unhealthy
7,overcast,64.0,65.0,"[64.0, 65.0]","[u'medium', u'moderate']",True,Play,1.5,medium,moderate
8,sunny,72.0,95.0,"[72.0, 95.0]","[u'high', u'unhealthy']",False,Don't Play,5.0,high,unhealthy
9,sunny,69.0,70.0,"[69.0, 70.0]","[u'high', u'good']",False,Play,5.0,high,good
10,rain,75.0,80.0,"[75.0, 80.0]","[u'medium', u'good']",False,Play,1.0,medium,good


# 4. Get feature names from cols2vec summary table
Now let's run cols2vec then reverse it using vec2cols.  In this case we will get feature names from the cols2vec summary table.

First run cols2vec:

In [21]:
%%sql
DROP TABLE IF EXISTS cols2vec_result, cols2vec_result_summary;
SELECT madlib.cols2vec(
    'golf',
    'cols2vec_result',
    'temperature, humidity',
    NULL,
    'id, temperature, humidity'
);
SELECT * FROM cols2vec_result ORDER BY id;

Done.
1 rows affected.
14 rows affected.


id,temperature,humidity,feature_vector
1,85.0,85.0,"[85.0, 85.0]"
2,80.0,90.0,"[80.0, 90.0]"
3,83.0,78.0,"[83.0, 78.0]"
4,70.0,96.0,"[70.0, 96.0]"
5,68.0,80.0,"[68.0, 80.0]"
6,65.0,70.0,"[65.0, 70.0]"
7,64.0,65.0,"[64.0, 65.0]"
8,72.0,95.0,"[72.0, 95.0]"
9,69.0,70.0,"[69.0, 70.0]"
10,75.0,80.0,"[75.0, 80.0]"


View the summary table with the feature_names dictionary:

In [22]:
%%sql
SELECT * FROM cols2vec_result_summary;

1 rows affected.


source_table,list_of_features,list_of_features_to_exclude,feature_names
golf,"temperature, humidity",None,"[u'temperature', u'humidity']"


Now use feature_names from the summary table above to name the columns of the split array:

In [23]:
%%sql
DROP TABLE IF EXISTS vec2cols_result;
SELECT madlib.vec2cols(
    'cols2vec_result',          -- source table containing the feature vector
    'vec2cols_result',          -- output table
    'feature_vector',           -- column with array entries to split
    (SELECT feature_names from cols2vec_result_summary),    -- feature_names from summary table of cols2vec
    'id'                        -- columns to keep from source table
);
SELECT * FROM vec2cols_result ORDER BY id;

Done.
1 rows affected.
14 rows affected.


id,temperature,humidity
1,85.0,85.0
2,80.0,90.0
3,83.0,78.0
4,70.0,96.0
5,68.0,80.0
6,65.0,70.0
7,64.0,65.0
8,72.0,95.0
9,69.0,70.0
10,75.0,80.0


This is the same as the format of the original 'golf' dataset that we started with.

# Drop columns
Drop some columns from the source table.

In [24]:
%%sql
DROP TABLE IF EXISTS dropcols_out;

SELECT madlib.dropcols(
    'golf',
    'dropcols_out',
    '"OUTLOOK", temperature, humidity'
);

SELECT * FROM dropcols_out ORDER BY id;

Done.
1 rows affected.
14 rows affected.


id,Temp_Humidity,clouds_airquality,windy,class,observation_weight
1,"[85.0, 85.0]","[u'none', u'unhealthy']",False,Don't Play,5.0
2,"[80.0, 90.0]","[u'none', u'moderate']",True,Don't Play,5.0
3,"[83.0, 78.0]","[u'low', u'moderate']",False,Play,1.5
4,"[70.0, 96.0]","[u'low', u'moderate']",False,Play,1.0
5,"[68.0, 80.0]","[u'medium', u'good']",False,Play,1.0
6,"[65.0, 70.0]","[u'low', u'unhealthy']",True,Don't Play,1.0
7,"[64.0, 65.0]","[u'medium', u'moderate']",True,Play,1.5
8,"[72.0, 95.0]","[u'high', u'unhealthy']",False,Don't Play,5.0
9,"[69.0, 70.0]","[u'high', u'good']",False,Play,5.0
10,"[75.0, 80.0]","[u'medium', u'good']",False,Play,1.0
